In [ ]:
#https://github.com/raghav1810/Music-analysis-and-Year-prediction/blob/master/Evolution%20of%20Pop%20Music.ipynb

In [11]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://master:7077") \
        .appName("whatever")\
        .getOrCreate()

spark_context = spark_session.sparkContext

In [9]:
# collect all directories where all h5 files are

from hdfs import InsecureClient
client = InsecureClient("http://master:50070", user="ubuntu")

paths = set()
for path, dirs, _files in client.walk("/home/ubuntu"):
    if not dirs and not path.endswith("AdditionalFiles"):
        paths.add(path)

In [20]:
def f(data):
    import h5py
    import io
    with io.BytesIO(data) as b, h5py.File(b, "r") as f:
        return f["metadata"]["songs"][0][9].decode() 

h5s = spark_context.union([spark_context.binaryFiles("hdfs://master:9000" + p) for p in paths])

from pprint import pprint
pprint(h5s.map(lambda tup: f(tup[1])).map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).takeOrdered(200, key=lambda x: -x[1]))

[('Mario Rosenstock', 13),
 ('Phil Collins', 12),
 ('Sugar Minott', 12),
 ('The Jackson Southernaires', 12),
 ('Aerosmith', 12),
 ('Nick Cave & The Bad Seeds', 11),
 ('Radiohead', 11),
 ('Snow Patrol', 11),
 ('Mikey Dread', 11),
 ('Line Renaud', 11),
 ('Bill & Gloria Gaither', 11),
 ('Selena', 10),
 ('White Heart', 10),
 ('Red Hot Chili Peppers', 10),
 ('Shakira', 10),
 ('The Del McCoury Band', 10),
 ('RUN-DMC', 10),
 ('Michael Jackson', 10),
 ('Jacques Douai', 10),
 ('Ayo', 10),
 ('Pumpkin Buzzard', 10),
 ('Stevie Ray Vaughan And Double Trouble', 10),
 ('Hank Locklin', 9),
 ('2 Minutos', 9),
 ('Madonna', 9),
 ('Skeeter Davis', 9),
 ('Van Halen', 9),
 ('The Marx Brothers', 9),
 ('Jimmy Riley', 9),
 ('Los Fabulosos Cadillacs', 9),
 ('Daniel Johnston', 9),
 ('DJ Nasty', 9),
 ('Michael Stanley Band', 9),
 ('Cali', 9),
 ('Sister Hazel', 9),
 ('R. Carlos Nakai', 9),
 ('Mariano Yanani', 9),
 ('Leroy Hutson', 9),
 ('Les Ogres De Barback', 9),
 ('Seal', 9),
 ('George Lopez', 9),
 ('Rihanna', 8

In [21]:
spark_context.stop()